# Bayesian Statistics

* Review
* Bayes Law
  - How to apply Bayes' law for Bayesian statistics
  - Prior/Likelihood/Posterior
* Using the Posterior
  - Credible sets
  - Posterior probabilities
* Finding the Posterior
  - (Some) Conjugate Pairs
  - Monte-Carlo Markov Chains

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import scipy.stats as st

from ipywidgets import interact, FloatSlider, IntSlider

%matplotlib inline

## Statistics Review

### What is a model?

A model is a probability distribution over a sequence of outcomes indexed by parameters.

Example: Let quarterly GDP growth be described by

$$y_{t} = 1.0 + \varepsilon$$

where $\varepsilon \sim N(0, 1)$

### What is a manifold of models

A manifold of models is a set of models with each model indexed by $\theta \in \Theta$

Example: Let quarterly GDP growth be described by

$$y_t = \theta + \varepsilon$$

with $\theta \in \Theta$ and $\Theta \equiv [-\infty, \infty]$

### What is a likelihood?

The **likelihood function** for a manifold of statistical models is a function that maps values of the parameters, $\theta \in \Theta$ and observations, $\tilde{y} \in \mathcal{Y}$, into the probability of having observed $\tilde{y}$ given $\theta$.

$$\mathcal{L}(\theta | \tilde{y}) \equiv f(\tilde{y} | \theta)$$

Example: Let quarterly GDP growth be described by

$$y_{t} = 1.0 + \varepsilon$$

where $\varepsilon \sim N(0, 1)$ then the likelihood of observing $\tilde{y}$ is given by

$$\mathcal{L}(\theta | \tilde{y}) = f(\tilde{y} | \theta) = \frac{1}{\sigma \sqrt{2 \pi}} \exp \left(\sum_i \frac{(\tilde{y}_i - \theta)^2}{2 \sigma^2}\right)$$

### Sampling distribution

Given observed data, $\tilde{y}$, a frequentist uses an estimator (and their corresponding properties) to produce a sampling distribution which specifies the distribution of the estimator, $\hat{\theta}$.

## Bayes' Law

At the core of Bayesian statistics is a simple rule known as Bayes' Law:

$$P(A | B) = \frac{P(B | A) P(A)}{P(B)}$$

In [ ]:
def bayes_law(p_bga, p_a, p_b):
    "Evaluates Bayes Law given P(B | A), P(A), and P(B)"
    return (p_bga * p_a) / p_b

**Obligatory medical test example**:

One of the examples that is almost always presented along with Bayes Law is the following:

Imagine that 0.001% of the population (1 in 100,000) has a particular medical condition. There is a test that can determine whether an individual has the medical condition with a false positive rate of 2% and a false negative rate of 1%.

\begin{align*}
  P(\text{has disease} | \text{positive test}) &= \frac{P(\text{positive test} | \text{has disease}) P(\text{has  disease})}{P(\text{positive test})} \\
  &= \frac{P(\text{positive test} | \text{has disease}) P(\text{has  disease})}{P(\text{positive test} | \text{has disease})P(\text{has disease}) + P(\text{positive test} | \text{no disease}) P(\text{no disease})} \\
  &= \frac{0.99 \times 0.00001}{0.99 \times 0.00001 + 0.02 \times 0.99999} \\
  &\approx 0.00049
\end{align*}

In [ ]:
prevalence = 0.02
false_negative = 0.25
false_positive = 0.008

p_bga = 1.0 - false_positive
p_a = prevalence
p_b = (
    # p(- | d) p(d) + p(- | nd) p(nd)
    prevalence*false_negative + (1.0 - prevalence)*(1.0 - false_positive)
)

bayes_law(p_bga, p_a, p_b)

### Bayes' law to describe data and parameters

Rather than a "manifold of models", a Bayesian begins with a "single model" described by

$$P(\theta, Y) = P(Y | \theta) P(\theta)$$

This "single model" is a joint distribution over the observed data and parameter values.

Consider using rules of conditional probabilities, we can rewrite this as:

$$\underbrace{P(\theta | Y)}_{\text{posterior}} = \frac{\overbrace{P(Y | \theta)}^{\text{likelihood}} \overbrace{P(\theta)}^{\text{prior}}}{\underbrace{P(Y)}_{\text{normalizing component}}}$$

#### Normalizing component: $P(Y)$

The normalizing component is often ignored because it's simply the value needed to ensure that the posterior is a probability distribution.

In fact, you will often find Bayes' Law written as $P(\theta | Y) \propto P(Y | \theta) P(\theta)$

#### Likelihood: $P(Y | \theta)$

We refer to this term as the likelihood component and establishes how likely it was to observe certain realizations of data for a particular parameter $\theta$

Corresponds to the likelihood function we've discussed previously.

#### Prior: $P(\theta)$

The prior is what most people would identify as the "defining feature" of Bayesian statistics.

It specifies the belief that the statistician assigns to different parameter values BEFORE seeing any data

**What is a prior?**

One often begins their inference with some ideas of what parameters make sense and which ones don't. A prior reflects the subjective beliefs of the statistician who is running the analysis (or reflects beliefs of the audience that they are speaking with!).

For example, in our model of quarterly GDP growth, we are relatively confident that the mean quarterly GDP growth in the US is not 100% so our prior would assign a low probability to $\theta = 100$.


**Doesn't a prior make analysis subjective?**

Yes. We are okay with that.

For why we think this isn't a problem, see [1](https://stat.columbia.edu/~gelman/research/published/philosophy_chapter.pdf), [2](https://statmodeling.stat.columbia.edu/2016/12/13/bayesian-statistics-whats/), [3](https://projecteuclid.org/euclid.ba/1340370426#toc), among many others...

#### Posterior: $P(\theta | Y)$

The posterior is the conditional distribution that describes the statistician's beliefs about the parameter values given the datat that they have observed.

All of Bayesian statistics will depend on finding the posterior.

## Using the Posterior

We will talk about how to find the posterior next, but, imagine that we had found it, then what could we use it for?

Suppose that the posterior, $P(\theta, \tilde{y})$, is given by $N(0, 1)$

In [ ]:
posterior = st.norm(0.0, 1.0)

**Credible sets**

An $\alpha$ credible set, $C_{\alpha}$, is a set such that $\text{Prob}(\theta \in C_{\alpha}) = \alpha$

**How to choose $C$?**

1. Highest posterior density interval: Choosing the narrowest interval that contains $\alpha$
2. Equal-tailed interval: Choose the interval where the probability of being below the interval is as likely as being above it

In [ ]:
def plot_credible_set(posterior, a, b, alpha=0.95):
    fig, ax = plt.subplots()

    # Plot the pdf
    min_x, max_x = posterior.ppf(0.001), posterior.ppf(0.999)
    x = np.linspace(min_x, max_x, 500)
    ax.plot(x, posterior.pdf(x))

    # Fill the credible interval
    csx = np.linspace(a, b, 500)
    ax.fill_between(csx, 0.0, posterior.pdf(csx), alpha=0.5)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_title(f"{alpha} Credible Set")

    return fig, ax

**Highest posterior density interval**

This interval is the solution to

\begin{align*}
  a, b \in &\arg \min (b - a) \\
  &\text{s.t.} \\
  &\int_a^b P(\theta | \tilde{y}) d\theta = \alpha
\end{align*}

In [ ]:
def hpd_credible_set(posterior, alpha):
    "Finds the highest posterior density interval credible set"
    # a s.t. \int_a^\inf P(theta | ytilde) dtheta = alpha
    min_a = posterior.ppf(0.001)
    max_a = posterior.ppf(0.999 - alpha)

    opt_me = lambda a: posterior.ppf(alpha + posterior.cdf(a)) - a
    sol = opt.minimize_scalar(opt_me, bounds=(min_a, max_a), method="bounded")

    astar = sol.x
    bstar = posterior.ppf(alpha + posterior.cdf(astar))

    return astar, bstar

In [ ]:
hpd_a, hpd_b = hpd_credible_set(posterior, 0.95)

plot_credible_set(posterior, hpd_a, hpd_b);

**Equal-tailed interval**:

This interval is the solution to

\begin{align*}
  a, b \text{ s.t } \\
  &\int_{-\infty}^{a} P(\theta | \tilde{y}) d\theta = \int_{b}^{\infty} P(\theta | \tilde{y}) d\theta = \frac{1 - \alpha}{2} \\
\end{align*}

In [ ]:
def et_credible_set(posterior, alpha):
    "Finds the equal-tailed interval credible set"
    astar = posterior.ppf((1.0 - alpha)/2)
    bstar = posterior.ppf(1.0 - (1.0 - alpha)/2)

    return astar, bstar

In [ ]:
et_a, et_b = et_credible_set(posterior, 0.95)

plot_credible_set(posterior, et_a, et_b);

In this case, the high posterior density interval and equal-tailed interval were the same because we had a symmetric, unimodal posterior... We could wind up with a non-symmetric posterior though and they would differ...

We use $C_{0.75}$ rather than $C_{0.95}$ to emphasize the differences.

In [ ]:
ns_posterior = st.lognorm(scale=np.exp(1.0), s=0.5)

ns_hpd_a, ns_hpd_b = hpd_credible_set(ns_posterior, 0.75)
ns_et_a, ns_et_b = et_credible_set(ns_posterior, 0.75)

In [ ]:
print(f"HPD CS ({ns_hpd_a:0.2f}, {ns_hpd_b:0.2f})")
print(f"ET CS ({ns_et_a:0.2f}, {ns_et_b:0.2f})")

In [ ]:
plot_credible_set(ns_posterior, ns_hpd_a, ns_hpd_b);

In [ ]:
plot_credible_set(ns_posterior, ns_et_a, ns_et_b);

**Beyond credible sets**

In addition to finding the credible sets, we can ask the posterior questions:

For example, we might be interested in knowing things like $\text{Prob}(\theta > 1)$ -- We can directly use the posterior to determine the answers.

In [ ]:
plot_credible_set(posterior, hpd_a, hpd_b);

In [ ]:
1.0 - posterior.cdf(1.0)

In [ ]:
plot_credible_set(ns_posterior, ns_hpd_a, ns_hpd_b);

In [ ]:
1.0 - ns_posterior.cdf(1.0)

## Finding the Posterior

The equation from Bayes' Law may make it look like it will be easy to determine the posterior...

Don't be fooled -- This is easier said than done!

**Ways to proceed**

1. Conjugate priors
2. Monte-Carlo Markov Chains
3. Case-by-case (we won't talk about this)

### Conjugate priors

If the prior, $P(\theta)$, and the posterior, $P(\theta | Y)$, belong to the same probability distribution family for a specified likelihood, $P(Y | \theta)$, then we say the prior is a conjugate prior for the likelihood.

Conjugate priors are convenient because they allow us to have a closed-form expression for the posterior


We will discuss a few commonly used conjugate priors, but there's a great [table of conjugate priors on Wikipedia](https://en.wikipedia.org/wiki/Conjugate_prior#Table_of_conjugate_distributions).

**Binomial likelihood with Beta prior**

This example should look familiar.

There is a known parameter $n$ which is the number of Bernoulli draws and an unknown parameter $p$ which specifies the probability of success for any Bernoulli trial.

* The prior is specified by the beta distribution with parameters $\alpha$ and $\beta$
* The likelihood is specified by the Binomial distribution with parameters $p$ and $n$


The distribution functions associated with the prior and likelihood are given by

* Likelihood, $P(k | p, n) = {n \choose{k}} p^k (1 - p)^{n-k}$
* Prior, $P(p) = \frac{p^{\alpha - 1} (1 - p)^{\beta - 1}}{B(\alpha, \beta)}$

Thus

\begin{align*}
  P(p | k, \alpha, \beta, n) &\propto {n \choose{k}} p^k (1 - p)^{n-k} \frac{p^{\alpha - 1} (1 - p)^{\beta - 1}}{B(\alpha, \beta)} \\
  &\propto p^{k + \alpha - 1} (1 - p)^{n - k + \beta - 1} \frac{{n \choose{k}}}{B(\alpha, \beta)} \\
  &\dots \\
\end{align*}


And, if one does the algebra, they can show that the posterior is given by $\text{Beta}(\alpha + k, \beta + (n-k))$

**Example**:

We'd like to know how likely a student is to pass their PhD qualifying exams.

* The PhD program only admits students that they think are likely to pass, so we begin with a prior $P(p) \sim \text{Beta}(8, 2)$
* The student takes 8 classes prior to taking the qualifying exam and passes each class with the same probability that they pass the qualifying exams
* The student has successfully passed 7 of their 8 classes

Since we are using a conjugate-prior, the posterior can be written as

$$P(\theta | k, \alpha, \beta, n) = \text{Beta}(\alpha + k, \beta + 1)$$

In [ ]:
# hyper parameters
alpha, beta = 8, 2
n = 16

# Data
k = 15

# Posterior
posterior = st.beta(alpha + k, beta + (n-k))

What is the 0.95 highest posterior density credible set for $p$?

In [ ]:
a_bb, b_bb = hpd_credible_set(posterior, 0.95)

plot_credible_set(posterior, a_bb, b_bb);

What is the probability that the student's success probability is above 0.85?

In [ ]:
1.0 - posterior.cdf(0.85)

What do we think the probability that the student succeeds is?

We could do this analytically (it turns out that it is just the mean of the distribution), but the computer is great at simulating... So let's go for the simulated answer

In [ ]:
ndraws = 1_000_000

np.mean(np.random.rand(ndraws) < posterior.rvs(ndraws))

In [ ]:
posterior.mean()

**Normal likelihood with Normal prior**

There is a known parameter $\sigma$ which is the standard deviation of the observed data and an unknown parameter $\mu$ which specifies the mean of $n$ observations, $\tilde{y}$.

* The prior is specified by the normal distribution with parameters $\mu_0$ and $\sigma_0$
* The likelihood is specified by the normal distribution with unknown parameter $\mu$ and known parameter $\sigma$


The posterior is given by

$$P(\mu | \tilde{y}, \mu_0, \sigma_0, \sigma) \sim N \left( \frac{1}{\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2}} \left( \frac{\mu_0}{\sigma_0^2} + \frac{\sum_{i=1}^n \tilde{y}_i}{\sigma^2} \right),  \frac{1}{\frac{1}{\sigma_0^2} + \frac{n}{\sigma^2}} \right)$$

**Example**

Let's return to our GDP growth model given by

$$y = \theta + \varepsilon$$

where $\varepsilon \sim N(0, 1)$. Suppose that our prior is given by $\theta \sim N(1, 4)$ and that we know the standard deviation of GDP growth is 1.5.

If we observe quarterly GDP growth from 1990 to 2015, what is our posterior over $\theta$?

In [ ]:
# Hyper parameters
mu_0, sigma_0 = 1.5, 2.0
sigma = 5.5

# Data
# US quarterly GDP growth from 1990 to 2015
ytilde = np.array([
    2.18, 1.49, 0.92, -0.17, 0.51, 1.52, 1.29, 0.94, 1.57, 1.69, 1.48, 1.74, 0.73, 1.18, 1.07, 1.91,
    1.45, 1.84, 1.16, 1.69, 0.9, 0.78, 1.35, 1.16, 1.23, 2.09, 1.23, 1.58, 1.25, 1.87, 1.69, 1.19, 1.15,
    1.16, 1.69, 1.9, 1.33, 1.14, 1.66, 2.25, 1.05, 2.45, 0.7, 1.16, 0.32, 1.19, -0.01, 0.6, 1.21, 0.97,
    0.91, 0.72, 1.01, 1.16, 2.25, 1.75, 1.28, 1.58, 1.61, 1.78, 1.91, 1.17, 1.8, 1.44, 2.04, 1.07, 0.86,
    1.22, 1.22, 1.22, 1.06, 1.01, -0.21, 1.06, 0.2, -1.86, -1.13, -0.29, 0.47, 1.44, 0.64, 1.39, 1.03, 1.07,
    0.3, 1.38, 0.62, 1.31, 1.41, 0.83, 0.65, 0.63, 1.29, 0.41, 1.27, 1.39, 0.13, 1.92, 1.66, 0.72, 0.86,
    1.22, 0.68, 0.17
])
n = len(ytilde)

var_post = 1.0 / ((1 / sigma_0**2) + (n / sigma**2))
mu_post = var_post * ((mu_0 / sigma_0**2) + (np.sum(ytilde) / sigma**2))

posterior = st.norm(mu_post, var_post)

a_nkm, b_nkm = hpd_credible_set(posterior, 0.95)
fig, ax = plot_credible_set(posterior, a_nkm, b_nkm)

mle_est = np.mean(ytilde)

ax.vlines(mle_est, 0.0, posterior.pdf(mle_est), linewidth=0.75, color="k", linestyle="--")
ax.annotate(
    "MLE Estimate", xy=(mle_est, posterior.pdf(mle_est)),
    xytext=(20, -10), textcoords="offset pixels",
    arrowprops={"arrowstyle": "->"}
)

**Multivariate normal likelihood with Normal-Inverse-Wishart prior**

There are unknown parameters $\mu$ and $\Sigma$.

* The prior over $\mu$ and $\Sigma$ are described by the [normal-inverse-Wishart distribution](https://en.wikipedia.org/wiki/Normal-Wishart_distribution) with parameters $\mu_0$, $\kappa_0$, $\nu_0$, and $\Psi$.
* The likelihood is specified by the multivariate normal distribution with unknown parameters $\mu$ and $\Sigma$


The posterior is then given by a normal-inverse-Wishart distribution with

\begin{align*}
  \mu' &= \frac{\kappa_0 \mu_0 + n \bar{y}}{\kappa_0 + n} \\
  \kappa' &= \kappa_0 + n \\
  \nu' &= \nu_0 + n \\
  \Psi' &= \Psi + \left( \sum_{i=1}^n (\tilde{y}_i - \bar{y}) (\tilde{y}_i - \bar{y})^T) \right) + \frac{\kappa_0 n}{\kappa_0 + n} (\bar{y} - \mu_0) (\bar{y} - \mu_0)^T
\end{align*}

**Example**

Let's return to our GDP growth model given but now add unemployment

$$\begin{bmatrix} y \\ u \end{bmatrix} = \begin{bmatrix} \theta_y \\ \theta_u \end{bmatrix} + \Sigma \varepsilon$$

where $\varepsilon \sim N(0, I)$.

Suppose that our prior is given by an normal-inverse-Wishart with parameters

\begin{align*}
  \mu_0 &= \begin{bmatrix} 1.0 & 0.0 \end{bmatrix} \\
  \kappa_0 &= 0.5 \\
  \nu_0 &= 5.0 \\
  \Psi_0 &= I
\end{align*}

If we observe quarterly GDP growth and the unemployment rate from 1990 to 2015, what is our posterior over $\theta \equiv \begin{bmatrix} \theta_y \\ \theta_u \end{bmatrix}$?

In [ ]:
# Hyper parameters
mu_0 = np.array([1.0, 0.0])
kappa_0 = 0.5
nu_0 = 5.0
Psi_0 = np.eye(2)

# Data
# US quarterly GDP growth and unemployment rate from 1990 to 2015
ytilde = np.array([
    [2.183031602838037, 5.3],
    [1.4869989124254746, 5.333333333333333],
    [0.9242909597068971, 5.7],
    [-0.17261512496183906, 6.133333333333334],
    [0.5070167149813454, 6.6000000000000005],
    [1.5191598325682953, 6.833333333333333],
    [1.290628057233878, 6.866666666666667],
    [0.9443054288176045, 7.1000000000000005],
    [1.573331800898381, 7.366666666666667],
    [1.6919577903984795, 7.6000000000000005],
    [1.4817108894267683, 7.633333333333333],
    [1.7385144094218319, 7.366666666666667],
    [0.72829568541386, 7.133333333333333],
    [1.1810756258415545, 7.066666666666666],
    [1.0744552124787754, 6.8],
    [1.912788803646781, 6.633333333333333],
    [1.4530625466762315, 6.566666666666666],
    [1.8449328325780945, 6.2],
    [1.1610984015164583, 6.0],
    [1.6943354146561118, 5.633333333333333],
    [0.8987043827146657, 5.466666666666666],
    [0.7804539283188694, 5.666666666666667],
    [1.347157900207585, 5.666666666666667],
    [1.164382982184975, 5.566666666666666],
    [1.2335920117191534, 5.533333333333334],
    [2.088996231540863, 5.5],
    [1.2270629067677286, 5.266666666666667],
    [1.57860729654693, 5.333333333333333],
    [1.2456035403402899, 5.233333333333333],
    [1.867469123143306, 5.0],
    [1.6903504884769882, 4.866666666666666],
    [1.1899585158325232, 4.666666666666667],
    [1.1473199521738087, 4.633333333333333],
    [1.164148568541301, 4.3999999999999995],
    [1.6878827260535578, 4.533333333333333],
    [1.8955395387199703, 4.433333333333334],
    [1.3263731372238352, 4.3],
    [1.1350218067583073, 4.266666666666667],
    [1.6558324562648608, 4.233333333333333],
    [2.246697327454572, 4.066666666666666],
    [1.0453080991553065, 4.033333333333333],
    [2.447520743323639, 3.9333333333333336],
    [0.7040228328775822, 4.0],
    [1.1550583464351227, 3.9],
    [0.32430231750570915, 4.233333333333333],
    [1.1930148338389124, 4.3999999999999995],
    [-0.014418056842246951, 4.833333333333333],
    [0.6039847516499552, 5.5],
    [1.2068897912196341, 5.7],
    [0.9663125760500213, 5.833333333333333],
    [0.9073911842490512, 5.733333333333333],
    [0.7224493408918731, 5.866666666666667],
    [1.0120071755647775, 5.866666666666667],
    [1.1567748828699376, 6.133333333333333],
    [2.2492160480868106, 6.133333333333333],
    [1.7458572534395422, .833333333333333],
    [1.2821010639992636, 5.7],
    [1.584021166142846, 5.599999999999999],
    [1.6050310401687584, 5.433333333333334],
    [1.7806953233837186, 5.433333333333334],
    [1.9078732753440297, 5.3],
    [1.1651208646868305, 5.1000000000000005],
    [1.8036452610227904, 4.966666666666667],
    [1.441412391339414, 4.966666666666667],
    [2.0372829446886787, 4.733333333333333],
    [1.072285492732128, 4.633333333333334],
    [0.8557429828464302, 4.633333333333334],
    [1.2241527275092645, 4.433333333333334],
    [1.2206184867838532, 4.5],
    [1.223163289702156, 4.5],
    [1.0612998712381305, 4.666666666666667],
    [1.007897968786109, 4.8],
    [-0.2074855970108369, 5.0],
    [1.055024152212014, 5.333333333333333],
    [0.1997621037051367, 6.0],
    [-1.8580419646884172, 6.866666666666667],
    [-1.1332498554384673, 8.266666666666667],
    [-0.28967219322705295, 9.299999999999999],
    [0.47002511696283555, 9.633333333333335],
    [1.4403918583731024, 9.933333333333332],
    [0.6380152175061715, 9.833333333333334],
    [1.3908235318092377, 9.633333333333333],
    [1.0305372509278765, 9.466666666666667],
    [1.067154414709326, 9.500000000000002],
    [0.29516083854417285, 9.033333333333333],
    [1.37618322016968, 9.066666666666668],
    [0.6173195228839967, 9.0],
    [1.3122881505401862, 8.633333333333333],
    [1.4135951979114436, 8.266666666666667],
    [0.8270973881128629, 8.2],
    [0.6494076957789918, 8.033333333333333],
    [0.6256446778405422, 7.8],
    [1.2881579850628988, 7.733333333333333],
    [0.4124121108360468, 7.533333333333334],
    [1.2671170673556276, 7.233333333333333],
    [1.3911360060699929, 6.933333333333334],
    [0.1253750994328673, 6.666666666666667],
    [1.91968747506146, 6.2],
    [1.656338595010154, 6.066666666666667],
    [0.7237190066369026, 5.7],
    [0.8598753876209342, 5.533333333333334],
    [1.2229801716887145, 5.433333333333334],
    [0.6796031317013007, 5.1000000000000005],
    [0.17102127410248968, 5.033333333333333],
])
n = ytilde.shape[0]

# Posterior
ybar = ytilde.mean(axis=0)
C = np.sum(
    [np.outer(ytilde[i, :] - ybar, ytilde[i, :] - ybar) for i in range(n)],
    axis=0
)
ybar_m_mu_sq = np.outer(ybar - mu_0, ybar - mu_0)

mu = (kappa_0*mu_0 + n*ybar)/(kappa_0 + n)
kappa = kappa_0 + n
nu = nu_0 + n
Psi = Psi_0 + C + ((kappa_0*n)/(kappa_0 + n))*ybar_m_mu_sq

`scipy.stats` doesn't have a normal-inverse-Wishart distribution, but we can use their `st.invwishart` and `st.multivariate_normal` to sample from the normal-inverse-Wishart by

1. Draw $\tilde{\Sigma} \sim W^{-1}(\nu, \Psi)$
2. Draw $\tilde{\mu} \sim N(\mu', \frac{1}{\kappa} \tilde{\Sigma})$

In [ ]:
ndraws = 5_000

mu_posterior = np.empty((ndraws, 2))
Sigma_posterior = np.empty((ndraws, 2, 2))

for i in range(ndraws):
    Sig_rvs = st.invwishart(df=nu, scale=Psi).rvs()
    Sigma_posterior[i, :, :] = Sig_rvs
    
    mu_rvs = st.multivariate_normal(mean=mu, cov=Sig_rvs/kappa).rvs()
    mu_posterior[i, :] = mu_rvs

In [ ]:
plt.hist(mu_posterior[:, 0], bins=25);

In [ ]:
plt.hist(mu_posterior[:, 1], bins=25);

In [ ]:
plt.hist(Sigma_posterior[:, 0, 0], bins=25);

In [ ]:
plt.hist(Sigma_posterior[:, 1, 1], bins=25);

In [ ]:
plt.hist(Sigma_posterior[:, 0, 1], bins=25);

### Monte-Carlo Markov Chains

We will discuss Monte-Carlo Markov Chains in more detail in a separate class, but we wanted to briefly touch on them to set the stage.

**Sampling from the posterior**

As we saw in our final example from the conjugate prior section, we don't need to be able to analytically compute the posterior in order to examine the posterior distribution.

Instead, if we can sample from the posterior distribution, that should be enough!

**How do we sample from the posterior when it is not a conjugate prior?**

In the previous section, we could only sample from the posterior because we _knew the distribution that we were sampling from_! What happens when we can't analytically compute this distribution?

Monte-Carlo Markov chain methods are going to build Markov chains that "happen" to have a stationary distribution that corresponds to the posterior distribution then by simulating the Markov chain, we are drawing from the posterior!

**How?**

Coming soon :)